In [7]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import vectorbt as vbt
import pandas as pd

In [9]:
close = pd.Series([1, 2, 3, 4, 5])

size = [1, 0, -1, 0, 1]
pf = vbt.Portfolio.from_orders(close, size, size_type='targetpercent')
pf.orders.records

,id,col,idx,size,price,fees,side
0,0,0,0,100.000000,1.0,0.0,0
1,1,0,1,100.000000,2.0,0.0,1
2,2,0,2,66.666667,3.0,0.0,1
3,3,0,3,66.666667,4.0,0.0,0
4,4,0,4,26.666667,5.0,0.0,0


In [15]:
import vectorbt as vbt
import pandas as pd

# Create sample price data
price = pd.Series([100, 105, 103, 108, 107, 110], 
                 index=pd.date_range('2023-01-01', periods=6))

# Define buy/sell signals
entries = pd.Series([True, False, True, False, False, False])  # Buy signals
exits = pd.Series([False, True, False, True, True, False])   # Sell signals

# Create portfolio
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100
)

# Analyze results
print("Final Portfolio Value:", pf.total_profit())
pf.plot().show()

Final Portfolio Value: 10.09708737864078


In [21]:
import vectorbt as vbt
import pandas as pd

# Create sample price data (7 days)
price = pd.Series([100, 102, 98, 95, 97, 103, 101],
                 index=pd.date_range('2023-01-01', periods=7))

# Define signals
long_entries =  [True, False, False, False, False, True, False]  # Buy signals
long_exits =    [False, True, False, False, False, False, True]  # Sell signals
short_entries = [False, False, True, False, False, False, False] # Short signals
short_exits =   [False, False, False, True, False, False, False] # Cover signals

# Create portfolio with long and short positions
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=long_entries,
    exits=long_exits,
    # short_entries=short_entries,
    # short_exits=short_exits,
    init_cash=1000,
    fees=0.001  # 0.1% trading fee
)

# Analyze results
print("=== Trade Summary ===")
print(pf.trades.records)
print("\n=== Final Portfolio Value ===")
print(f"${pf.total_profit():.2f}")

=== Trade Summary ===
   id  col      size  entry_idx  entry_price  entry_fees  exit_idx  \
0   0    0  9.990010          0        100.0    0.999001         1   
1   1    0  9.873253          5        103.0    1.016945         6   

   exit_price  exit_fees        pnl    return  direction  status  parent_id  
0       102.0   1.018981  17.962038  0.017980          0       1          0  
1       101.0   0.997199 -21.760650 -0.021398          0       1          1  

=== Final Portfolio Value ===
$-3.80


In [22]:
# Plot portfolio with positions
fig = pf.plot()
fig.show()

In [ ]:
import vectorbt as vbt

price = vbt.YFData.download('BTC-USD').get('Close')

pf = vbt.Portfolio.from_holding(price)
pf.total_profit()

np.float64(17104.55480847657)

In [11]:
fast_ma = vbt.MA.run(price, 10)
slow_ma = vbt.MA.run(price, 50)
entries = fast_ma.ma_crossed_above(slow_ma) #buy True - we buy
exits = fast_ma.ma_crossed_below(slow_ma)   #sell - we sell

pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=100)
print(pf.total_profit())
fig = pf.plot(subplots=['orders', 'trade_pnl', 'cum_returns'])
fig.show()

32519.690205381277


In [ ]:
import numpy as np

symbols = ["BTC-USD", "ETH-USD"]
price = vbt.YFData.download(symbols, missing_index='drop').get('Close')

n = np.random.randint(1, 1001, size=1000).tolist()
pf = vbt.Portfolio.from_random_signals(price, n=n, init_cash=100, seed=42)

mean_expectancy = pf.trades.expectancy().groupby(['randnx_n', 'symbol']).mean()
fig = mean_expectancy.unstack().vbt.scatterplot(xaxis_title='randnx_n', yaxis_title='mean_expectancy')
fig.show()

/home/roman/Desktop/binance_ai/.venv/lib/python3.12/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


In [2]:
import vectorbt as vbt
import numpy as np

symbols = ["BTC-USD", "ETH-USD", "LTC-USD"]
price = vbt.YFData.download(symbols, missing_index='drop').get('Close')

windows = np.arange(2, 101)
fast_ma, slow_ma = vbt.MA.run_combs(price, window=windows, r=2, short_names=['fast', 'slow'])
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')
pf = vbt.Portfolio.from_signals(price, entries, exits, **pf_kwargs)

fig = pf.total_return().vbt.heatmap(
    x_level='fast_window', y_level='slow_window', slider_level='symbol', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

/home/roman/Desktop/binance_ai/.venv/lib/python3.12/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)
/home/roman/Desktop/binance_ai/.venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

/home/roman/Desktop/binance_ai/.venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

/home/roman/Desktop/binance_ai/.venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range flo

In [8]:
symbols = ["BTC-USD", "ETH-USD", "ADA-USD"]
price = vbt.YFData.download(symbols, period='6mo', missing_index='drop').get('Close')
bbands = vbt.BBANDS.run(price)

def plot(index, bbands):
    bbands = bbands.loc[index]
    fig = vbt.make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.15,
        subplot_titles=('%B', 'Bandwidth'))
    fig.update_layout(template='vbt_dark', showlegend=False, width=750, height=400)
    bbands.percent_b.vbt.ts_heatmap(
        trace_kwargs=dict(zmin=0, zmid=0.5, zmax=1, colorscale='Spectral', colorbar=dict(
            y=(fig.layout.yaxis.domain[0] + fig.layout.yaxis.domain[1]) / 2, len=0.5
        )), add_trace_kwargs=dict(row=1, col=1), fig=fig)
    bbands.bandwidth.vbt.ts_heatmap(
        trace_kwargs=dict(colorbar=dict(
            y=(fig.layout.yaxis2.domain[0] + fig.layout.yaxis2.domain[1]) / 2, len=0.5
        )), add_trace_kwargs=dict(row=2, col=1), fig=fig)
    return fig

vbt.save_animation('bbands.gif', bbands.wrapper.index, plot, bbands, delta=90, step=3, fps=3)

  0%|          | 0/31 [00:00<?, ?it/s]